In [1]:
import pandas as pd
import re
import os

In [2]:
erp_data_path = list(os.walk('./Data_input/erp数据'))[0][0] + '/' + list(os.walk('./Data_input/erp数据'))[0][2][0]
erp_ptsku = pd.read_csv(erp_data_path, dtype=str).fillna('')

In [3]:
# 定义oe清洗函数
def normalise_oe(oe):
    oe = str(oe).strip() #清空前后空字符串
    oe = re.sub(r'[\u4e00-\u9fa5]+[a-zA-Z]*', '', oe) #去除汉字及后面所有字母        
    oe = re.sub('\（', '(', oe) #将所有中文"（"符号替换为英文"("
    oe = re.sub('\）', ')', oe) #将所有中文"）"符号替换为英文")"
    oe = re.sub('\\([^)]*\\)', '', oe)  #删除所有"()"及其内部的字符（因为通常不是OE号）
    oe = re.sub('\\*+', '', oe) #删除所有的"*"

    oe = re.sub('  +', ';', oe) #将两个以上的空格替换为';'
    oe = re.sub(' ', '', oe) #删除单空格
    oe = re.sub('/+', '', oe) #删除'/'
    oe = re.sub('\n', ';', oe) #将换行符替换为';'
    oe = re.sub('[；：，,、+]', ';', oe) #将[；：，,、]替换为';'
    oe = re.sub('\u200e', '', oe) #清除不可见字符
    oe = re.sub(';$', '', oe) # 匹配末尾的';'

    #去除空值及纯字母
    list1 = [i for i in str(oe).split(';') if i != '' and i.isalpha() == False and not re.fullmatch(r'[a-zA-Z]+', i) == True]
    list1 = [re.sub(r'-\d+[PCSpcs].*', '', i) for i in list1] # 将每个oe中后缀"-数字pcs"及之后的内容移除
    list1 = [re.sub(r'with.*', '', i).strip() for i in list1] # 将每个oe中'with'之后内容移除
    
    # 对于每个oe,如果存在"-"且每个"-"后的字符数大于3且均为英文和空格组合，则删掉（若匹配多项只删最后一项,删掉部分'-'之后内容必须大2）
    list2 = []
    for i in list1:
        if '-' in i and len(i[i.index('-')+1:]) > 3:
            try:
                if len(re.compile('-[a-zA-Z ]+$').findall(i)[0]) > 3:
                    list2.append(re.sub('-[a-zA-Z ]+$', '', i))
                else:
                    list2.append(i)
            except:
                list2.append(i)
        else:
            list2.append(i)
    
    list2 = [re.sub(r'\-', '', i) for i in list2] #清除"-"
    list2 = [re.sub(r'^[^0-9]+[:\.]', '', i) for i in list2] #去除":"及前面所有字母（英文冒号）
    list2 = [re.sub(r'\.+', '', i) for i in list2] # 去除"."
    list2 = [re.sub(r'[:]+', ';', i) for i in list2] # ":"替换为";"（英文冒号）
    list2 = [i for i in list2 if i != '' and i.isalpha() == False]  #去除空值
    list2 = [re.sub(r'\r','',i) for i in list2]
    list2 = [i for i in list2 if len(i)>2] # 去除一些1位数和2位数字符串
    list2 = [oe.upper() for oe in list2] # 将OE中的字母全部转换为大写
    
    list3 = list(set(list2)) #去重
    list3.sort(key = list2.index)# 去重后保持OE号原来的顺序
    oe=';'.join(list3).replace(';;', ';')
    try:
        if oe[0] == ';':
            oe = oe[1:]
        elif oe[-1] == ';':
            oe = oe[:-1]
    except:
        pass
    return oe.upper()

In [4]:
# oe清洗
erp_ptsku['OE_original'] = erp_ptsku['OE'].astype(str)
erp_ptsku['OE'] = [normalise_oe(erp_ptsku.loc[i, 'OE']) for i in range(len(erp_ptsku))]
erp_ptsku['OE'] = erp_ptsku['OE'].astype(str)
# erp_ptsku

In [5]:
# 筛选
erp_ptsku = erp_ptsku[erp_ptsku['是否采购']=='是'].copy()
erp_ptsku = erp_ptsku[erp_ptsku['SKU状态']=='Active'].copy()
# erp_ptsku = erp_ptsku[sku_data['SKU类型']!='工单sku'].copy()
erp_ptsku_clean = erp_ptsku[erp_ptsku['SKU'].str.startswith('DEB')==False].copy()
# erp_ptsku_clean

In [6]:
#分割商品分类中英文
# 中文部分
erp_ptsku_clean['商品分类(中文)'] = [i.split('/')[-1] for i in erp_ptsku_clean['商品分类']]
# 英文部分：注意部分英文品名中包含'/'(例：A/C Hose)
en_sku_list = [i.split('/')[:-1] for i in erp_ptsku_clean['商品分类']]
for i in range(len(en_sku_list)):
    en_sku_list[i] = '/'.join(en_sku_list[i])
erp_ptsku_clean['商品分类(英文)'] = en_sku_list
# erp_ptsku_clean[['商品分类','商品分类(中文)','商品分类(英文)']]

In [7]:
erp_ptsku_clean.to_csv('./Data_output/【已清洗】PTSKU_erp.csv', index=False)
erp_ptsku_clean

,SKU,GID,子市场,SKU类型,是否采购,商品层级,商品状态,供应商SKU,EAN,OE,...,运营专员,物控专员,客服专员,品控专员,SKU状态,包材规格,包装注意事项,OE_original,商品分类(中文),商品分类(英文)
0,IC10738-FC,IC10738,"FR,APM,DE,APM-VC,IT,UK,APM-UK",采购SKU,是,普通,正常,CR-IGN170B,,12137510738;12131487707,...,,,,,Active,,,12137510738;12131487707,点火线圈,Ignition Coil
1,IC08063-FC,IC08063,"FR,AU,APM,DE,APM-VC,IT",采购SKU,是,普通,正常,O-IGN095,,1208063;10457075;1103905;1103872;DR44,...,,,,,Active,,,1208063;10457075;1103905;1103872;DR44;DR-44(Bl...,点火线圈,Ignition Coil
2,WLA73401ZX,WLA73401,"APM,DE",采购SKU,是,普通,正常,KK25-5934,,1273401;1273403;1273420;93171541;93185524;9318...,...,,,,,Active,,,1273401;1273403;1273420;93171541;93185524;9318...,雨刮连杆总成,Wiper Linkage Assembly
3,WIM20670ZX,WIM20670,"AP-CA,US-CA,US,AP,YHT",采购SKU,是,TOP20,重点维护,KK17-1867,,8C3Z17508;9L1417508;9C3417504;WM748;402067,...,,,,,Active,,,8C3Z-17508-C;9L14-17508-AB;9C34-17504-AB;WM-74...,雨刮电机,Wiper Motor
4,WIM02013-C1,WIM02013,"AP-CA,US-CA,US,AP",采购SKU,是,普通,正常,KK17-1834C,,6C2Z17508;F87Z17508;F8UU17504;ZZP067350;402013,...,,,,,Active,,,6C2Z-17508-AARM;F87Z-17508-AB;F8UU-17504-AA;ZZ...,雨刮电机,Wiper Motor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579274,SGAU002,SGAU002,"AP-CA,US-CA,APM,DE,AP,US,YHT,FR,IT",采购SKU,是,普通,正常,AU2014,,20072015;Q7,...,,,,,Active,,,2007-2015 奥迪 Q7 运动版,挡泥板,Splash Guard
579275,SGAU001,SGAU001,"AP-CA,US-CA,APM,DE,AP,US,YHT,IT,FR",采购SKU,是,普通,正常,AU2013,,20072015;Q7,...,,,,,Active,,,2007-2015 奥迪 Q7 普通版,挡泥板,Splash Guard
579278,SGAC005,SGAC005,"AP-CA,US-CA,AP,US,YHT",采购SKU,是,TOP20,正常,OC2002,,RDX20182019,...,,,,,Active,,,讴歌 RDX 2018-2019,挡泥板,Splash Guard
579280,SGAC003,SGAC003,"US,YHT,AP",采购SKU,是,普通,正常,OC1001,,20132016,...,,,,,Active,,,2013-2016 讴歌 TL,挡泥板,Splash Guard
